In [2]:
import pandas as pd
import numpy as np
from math import cos, sin, sqrt
from spatial_rear_intersection import spatialRearIntersection

def rotationMatrix(phi, omega, kappa):
    R = np.array([
            [cos(phi)*cos(kappa)-sin(phi)*sin(omega)*sin(kappa), -cos(phi)*sin(kappa) - sin(phi)*sin(omega)*cos(kappa), -sin(phi)*cos(omega)], 
            [cos(omega)*sin(kappa), cos(omega)*cos(kappa), -sin(omega)], 
            [sin(phi)*cos(kappa) + cos(phi)*sin(omega)*sin(kappa), -sin(phi)*sin(kappa) + cos(phi)*sin(omega)*cos(kappa), cos(phi)*cos(omega)]
        ])
    
    return R


In [3]:
df = pd.read_excel('input2.xlsx', engine='openpyxl', dtype=float)
data = df.to_numpy()

GCP_left = data[0:4, 0:2]
GCP_right = data[0:4, 2:4]
GCP_ground = data[0:4, 4:7]
unknownPoint_left = data[4:9, 0:2]
unknownPoint_right = data[4:9, 2:4]

m = 10000
f = 150

print(GCP_left)
print(GCP_right)
print(unknownPoint_left[0,0])
print(unknownPoint_left[0,1])

[[ 16.012  79.963]
 [ 88.56   81.134]
 [ 13.362 -79.37 ]
 [ 82.24  -80.027]]
[[-73.93   78.706]
 [ -5.252  78.184]
 [-79.122 -78.879]
 [ -9.887 -80.089]]
51.758
80.555


In [8]:
exteriorOrientation_left = spatialRearIntersection(m, f, GCP_left, GCP_ground)
exteriorOrientation_right = spatialRearIntersection(m, f, GCP_right, GCP_ground)

print(exteriorOrientation_left)
print(exteriorOrientation_right)


[4.99977015e+03 4.99972886e+03 2.00000234e+03 2.15007692e-04
 2.90644338e-02 9.52470639e-02 6.57792163e-03]
[5.89682856e+03 5.07024426e+03 2.03044324e+03 1.44335502e-02
 4.60182911e-02 1.10469036e-01 7.79815109e-03]


In [5]:
R_left = rotationMatrix(exteriorOrientation_left[3], exteriorOrientation_left[4], exteriorOrientation_left[5])
R_right = rotationMatrix(exteriorOrientation_right[3], exteriorOrientation_right[4], exteriorOrientation_right[5])

auxiliary_left = R_left @ np.array([GCP_left[1,0], GCP_left[1,1], -150])
auxiliary_right = R_right @ np.array([GCP_right[1,0], GCP_right[1,1], -150])

print(auxiliary_left)
print(auxiliary_right)


[  80.47417752   93.50996941 -147.32749889]
[ -11.72644406   83.94708052 -146.47764178]


In [6]:
B = exteriorOrientation_right[0:3] - exteriorOrientation_left[0:3]

print(B)

N_1 = (B[0]*auxiliary_right[2] - B[2]*auxiliary_right[0]) / (auxiliary_left[0]*auxiliary_right[2] - auxiliary_right[0]*auxiliary_left[2])
N_2 = (B[0]*auxiliary_left[2] - B[2]*auxiliary_left[0]) / (auxiliary_left[0]*auxiliary_right[2] - auxiliary_right[0]*auxiliary_left[2])

print(N_1)
print(N_2)

[897.05840477  70.5153993   30.44090122]
9.695832169875514
9.959906417521903


In [7]:


ground_1 = exteriorOrientation_left[0:3] + N_1 * auxiliary_left
ground_2 = exteriorOrientation_right[0:3] + N_2 * auxiliary_right

print(ground_1)
print(ground_2)

[5780.03427279 5906.38582716  571.5396373 ]
[5780.03427279 5906.34932276  571.5396373 ]
